# Input and Output State

Quando abbiamo creato il nostro advanced RAG Agent abbiamo notato che l'agente restituisce molta informazione per gli step intermedi come "proceed_to_generate" o "rephrase_count". Questa informazione va bene per il debugging, ma in generale per la maggior parte delle volte siamo solo interessati nella risposta finale dell'LLM.

LangGraph ci consente di fare questo (avere solo la risposta finale dell'LLM) definendo input e output state.

Vediamo come funziona in pratica.

In [42]:
from dotenv import load_dotenv

load_dotenv()

True

Prima definiamo un workflow come prima dove otteniamo troppa informazione più di quanta ci è necessaria. E poi andiamo a ripulire tale info.

In [43]:
from langchain_openai import ChatOpenAI
from langchain_ollama.chat_models import ChatOllama

# model = ChatOpenAI(model="gpt-4o")
model = ChatOllama(base_url="http://127.0.0.1:11434", model="qwen2.5:7b")

In [44]:
from typing import TypedDict

class ChatMessages(TypedDict):
    question: str
    answer: str
    llm_calls: int

In [45]:
def call_model(state: ChatMessages):
    question = state['question']
    llm_calls = state.get('llm_calls', 0)
    state['llm_calls'] = llm_calls + 1
    print("LLM_CALLS:", state['llm_calls'])
    response = model.invoke(input=question)
    state['answer'] = response.content
    return state

In [46]:
from langgraph.graph import StateGraph, START, END  

workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [47]:
graph.invoke(input={"question": "Whats the highest mountain i the world?"})

LLM_CALLS: 1


{'question': 'Whats the highest mountain i the world?',
 'answer': 'The highest mountain in the world is Mount Everest, which stands at an elevation of 8,849 meters (29,031.69 feet) above sea level. It is located in the Mahalangur Himal sub-range of the Himalayas, on the border between Nepal and Tibet (China).',
 'llm_calls': 1}

Noi siamo solo interessati alla risposta del LLM, quindi dobbiamo scartare "question" e "llm_calls".

Per fare questo dobbiamo definire più state objects.

Vediamo che definiamo 4 stati. 

Abbiamo l'input state, private state (o state intermedio) e output state.

Creiamo anche un Overall state object che eredita tutti gli stati, ma non aggiunge nessuna chiave.

L'input state passa solo la question all'LLM.

Lo stato intermedio deve sapere il numero di LLM calls che sono state fatte.

Ma per l'output state siamo solo interessati alla risposta.

In [48]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

class InputState(TypedDict):
    question: str

class PrivateState(TypedDict):
    llm_calls: int

class OutputState(TypedDict):
    answer: str

# full state object con tutti gli stati 
class OverallState(InputState, PrivateState, OutputState):
    pass


In [49]:
# passiamo il nostro full state object e 
# altri due parametri input e output dove passiamo gli stati di input e output
# rispettivamente
workflow = StateGraph(OverallState, input=InputState, output=OutputState)

workflow.add_edge(START, "agent")

workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()


In [50]:
# passiamo solo lo stato di input 
# e ci restiturà solo lo stato di output ovvero "answer"
graph.invoke({"question": "Whats the highest mountain in the world?"})

LLM_CALLS: 1


{'answer': 'The highest mountain in the world is Mount Everest, which stands at an elevation of 8,848.86 meters (29,031.7 feet) above sea level. It is located in the Mahalangur Himal sub-range of the Himalayas on the border between Nepal and Tibet (China).'}

Dunque definendo gli stati di input e di output possiamo pulire lo stato ed ottenere solo la parte che ci interessa.

# Add Runtime configuration

Vediamo come aggiungere la runtime configuration.

Questa può essere necessaria se vogliamo cambiare LLM al volo senza compilazione del grafo, o se abbiamo users da diversi paesi e vogliamo rispondere agli users nella loro lingua, questo dovrebbe essere fatto senza compilare il grafo.

### 🔹 **Cos'è `RunnableConfig` in LangChain?**
`RunnableConfig` è una **configurazione opzionale** che puoi passare quando esegui un `Runnable` in LangChain. Serve per **personalizzare il comportamento dell'esecuzione**, come:

- **Tracing e Logging** (per il debug e il monitoraggio)
- **Gestione di callbacks** (per ricevere notifiche sugli eventi)
- **Timeouts** (per evitare esecuzioni troppo lunghe)
- **Modifica dei parametri di runtime** (batch processing, streaming, ecc.)

---

## **📌 Esempio base di `RunnableConfig`**
```python
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig

# Funzione che simula un task
def double(x):
    return x * 2

# Creiamo un RunnableLambda
runnable = RunnableLambda(double)

# Config personalizzata con un nome di esecuzione
config = RunnableConfig(tags=["doubling_function"])

# Eseguiamo con la configurazione
result = runnable.invoke(10, config=config)

print(result)  # Output: 20
```
✅ **Qui `tags=["doubling_function"]` permette di tracciare l'esecuzione per debugging.**

---

## **🔹 Parametri principali di `RunnableConfig`**
`RunnableConfig` supporta diversi parametri, tra cui:

### **1️⃣ `tags` - Etichette per tracciare l'esecuzione**
```python
config = RunnableConfig(tags=["classification_pipeline"])
```
💡 Utile per tracciare le esecuzioni in logging e debugging.

---

### **2️⃣ `metadata` - Informazioni aggiuntive**
```python
config = RunnableConfig(metadata={"user_id": 123, "experiment": "test_1"})
```
💡 Può contenere qualsiasi informazione utile per analisi o monitoraggio.

---

### **3️⃣ `max_concurrency` - Numero massimo di esecuzioni simultanee**
```python
config = RunnableConfig(max_concurrency=5)
```
💡 Utile per eseguire più istanze dello stesso `Runnable` in parallelo.

---

### **4️⃣ `callbacks` - Aggiunta di callback per il monitoraggio**
Puoi usare i **callback** per monitorare quando un `Runnable` inizia o termina l'esecuzione.

```python
from langchain_core.callbacks import BaseCallbackHandler

class MyCallbackHandler(BaseCallbackHandler):
    def on_start(self, *args, **kwargs):
        print("Runnable started!")

    def on_end(self, *args, **kwargs):
        print("Runnable finished!")

config = RunnableConfig(callbacks=[MyCallbackHandler()])
```
💡 Ora, ogni volta che esegui il `Runnable`, verranno stampati i messaggi `"Runnable started!"` e `"Runnable finished!"`.

---

## **📌 Applicazione avanzata: `RunnableConfig` con `RunnableSequence`**
Se hai una pipeline di più `Runnable`, puoi passare `RunnableConfig` per monitorare l'intera esecuzione.

```python
from langchain_core.runnables import RunnableLambda, RunnableSequence

# Definiamo due funzioni
def add_one(x):
    return x + 1

def multiply_by_two(x):
    return x * 2

# Creiamo una sequenza di Runnable
pipeline = RunnableSequence(
    RunnableLambda(add_one),
    RunnableLambda(multiply_by_two)
)

# Configurazione con logging
config = RunnableConfig(tags=["math_pipeline"])

# Eseguiamo la sequenza con la config
result = pipeline.invoke(10, config=config)

print(result)  # Output: (10 + 1) * 2 = 22
```
✅ **L'intera pipeline è tracciata sotto il tag `math_pipeline`.**

---

## **📌 Quando usare `RunnableConfig`?**
✅ **Debugging & Tracing** → Usa `tags` e `metadata` per analizzare l'esecuzione.  
✅ **Parallelismo** → Controlla con `max_concurrency`.  
✅ **Monitoraggio** → Usa `callbacks` per ricevere notifiche sugli eventi.  

💡 **Conclusione**: `RunnableConfig` è utile per personalizzare e monitorare il comportamento dei `Runnable` in LangChain, rendendolo più flessibile e scalabile. 🚀

In [51]:
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables.config import RunnableConfig
from langchain.schema import SystemMessage, HumanMessage

# se vogliamo aggiungere una configurazione runtime, dobbiamo passare 
# il parametro config alla funzione che definisce il nodo
# Per rendere il nostro grafo configurabile, ogni informazione di configurazione
# deve essere passata dentro la "configurable" key
# qui possiamo definire dizionari con chiavi e valori
def call_model(state: OverallState, config: RunnableConfig):
    # vediamo come configurare la lingua
    language = config['configurable'].get("language", "English")
    # utilizziamo il valore della lingua preso dal dizinario config['configurable']
    # nel nostro system_prompt per far si che il modello risponda in una lingua dinamica
    system_message_content = "Respond in {language}".format(language=language)

    system_message = SystemMessage(content=system_message_content)

    messages = [system_message, HumanMessage(content=state['question'])]

    response = model.invoke(messages)

    return {"answer": response}


In [52]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [53]:
# usiamo la configurazione definendo un oggetto config
# e passandolo nel metodo invoke()
config = {"configurable": {"language": "Spanish"}} 

graph.invoke(input={"question": "What's the highest mountain in the world?"}, config=config)

{'question': "What's the highest mountain in the world?",
 'answer': AIMessage(content='El Monte Everest es el monte más alto del mundo, con una altura de 8.849 metros sobre el nivel del mar.', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-12T09:47:20.5051008Z', 'done': True, 'done_reason': 'stop', 'total_duration': 744974200, 'load_duration': 60515300, 'prompt_eval_count': 25, 'prompt_eval_duration': 18000000, 'eval_count': 30, 'eval_duration': 660000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-e0a12e32-8b9d-4e3c-ad29-6264d2dbdc35-0', usage_metadata={'input_tokens': 25, 'output_tokens': 30, 'total_tokens': 55})}